In [1]:
import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
os.environ['OPENAI_API_KEY']=''

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.callbacks import get_openai_callback

In [5]:
import pandas as pd

In [6]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [7]:
df_file=pd.read_csv(file)
df_file.head(1)

,Unnamed: 0,name,description
0,0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...


In [8]:
df_file.head(1).values

array([[0, "Women's Campside Oxfords",
        'This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.']],
      dtype=object)

In [9]:
docs = loader.load()

In [10]:
type(docs[0])

langchain.schema.Document

In [11]:
from langchain.embeddings import OpenAIEmbeddings

In [12]:
with get_openai_callback() as cb:
    embeddings = OpenAIEmbeddings()
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [13]:
with get_openai_callback() as cb:
    db = DocArrayInMemorySearch.from_documents(docs,embeddings)
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [14]:
query = "Please suggest a shirt with sunblocking"

In [15]:
docs = db.similarity_search(query)

In [16]:
len(docs)

4

In [17]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [18]:
llm = ChatOpenAI(temperature = 0.0)
retriever = db.as_retriever()

In [19]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [20]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [21]:
with get_openai_callback() as cb:
    response = qa_stuff.run(query)
    print(cb)



> Entering new RetrievalQA chain...

> Finished chain.
Tokens Used: 1269
	Prompt Tokens: 730
	Completion Tokens: 539
Successful Requests: 1
Total Cost (USD): $0.002538
